In [9]:
"""
greenhouse_analytics.py

Full pipeline for Smart Greenhouse Environmental Analytics:
 - the first step involves in the process of ingesting raw hourly sensor data
 - the second step is cleaning & resampling
 - implementing Exploratory Data Analysis (EDA) & getting summary stats
 - Define ideal ranges & compute KPIs
 - Anomaly detection (rule-based, z-score, IsolationForest)
 - Cross-correlation checks
 - Short-term forecasting (Prophet preferred; RandomForest fallback)
 - Export cleaned data, anomalies, forecast, and brief report skeleton

Author: Harish Ragavendra Chittibabu
Date: 2024-12-16
"""

# ----------------------------
# 01) Imports & config
# ----------------------------
import os
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as matplt
import seaborn as sns
import joblib
from datetime import timedelta

from sklearn.ensemble import IsolationForest, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy import stats
from statsmodels.tsa.stattools import ccf

# Prophet may be installed as 'prophet' or 'fbprophet'
try:
    from prophet import Prophet
    PROPHET_AVAILABLE = True
except Exception:
    try:
        from fbprophet import Prophet
        PROPHET_AVAILABLE = True
    except Exception:
        PROPHET_AVAILABLE = False

# ----------------------------
# 1) Paths & parameters
# ----------------------------
RAW_PATH = 'raw/greenhouse_raw.csv'
CLEANED_PATH = 'data/greenhouse_cleaned.csv'
ANOMALIES_PATH = 'data/anomalies.csv'
FORECAST_PATH = 'data/temperature_forecast.csv'
MODEL_PATH = 'models/temp_forecast.pkl'
REPORT_PATH = 'report/findings.md'
PLOTS_DIR = 'images/'

IDEAL_RANGES = {
    'temperature': (20.0, 28.0),    # °C
    'humidity': (60.0, 80.0),       # %
    'soil_moisture': (30.0, 60.0),  # %
    'co2': (400.0, 1000.0),         # ppm
}

os.makedirs('data', exist_ok=True)
os.makedirs('models', exist_ok=True)
os.makedirs('report', exist_ok=True)
os.makedirs(PLOTS_DIR, exist_ok=True)

# ----------------------------
# Helper: safe column check
# ----------------------------
def assert_columns(df, cols):
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing expected columns in data: {missing}")

# ----------------------------
# 2) Ingest data
# ----------------------------
def ingest(path=RAW_PATH):
    print('Ingesting data from', path)
    df = pd.read_csv(path, parse_dates=['timestamp'])
    df = df.sort_values('timestamp').reset_index(drop=True)
    print('Rows:', len(df))
    return df

# ----------------------------
# 3) Basic cleaning & resampling
# ----------------------------
def clean_and_resample(df, freq='1H'):
    # Ensure expected columns exist
    expected_cols = ['timestamp', 'temperature', 'humidity', 'soil_moisture', 'co2', 'light']
    assert_columns(df, expected_cols)

    # Drop duplicates by timestamp (keep last)
    df = df.drop_duplicates(subset='timestamp', keep='last').copy()
    df = df.set_index('timestamp').sort_index()

    # Ensure numeric columns
    for c in ['temperature','humidity','soil_moisture','co2','light']:
        df[c] = pd.to_numeric(df[c], errors='coerce')

    # Resample to hourly regular grid (mean aggregation)
    df = df.resample(freq).mean()

    # Interpolate small gaps (<= 3 periods) using time interpolation
    df.interpolate(method='time', limit=3, inplace=True)

    # Optionally drop rows still full-NaN or mark them
    df = df.dropna(how='all')

    # Remove obviously invalid readings
    df = df[(df['temperature'].between(-20, 60) | df['temperature'].isna())]
    df = df[(df['humidity'].between(0, 100) | df['humidity'].isna())]

    df.to_csv(CLEANED_PATH)
    print('Saved cleaned data to', CLEANED_PATH)
    return df

# ----------------------------
# 4) Resample aggregates (daily/weekly) & summary
# ----------------------------
def aggregates_and_summary(df):
    daily = df.resample('D').agg(['mean','min','max','std'])
    weekly = df.resample('W').mean()
    summary = df.describe().T
    # Save a basic summary plot
    plt.figure(figsize=(10,6))
    df[['temperature','humidity']].resample('D').mean().plot()
    plt.title('Daily mean Temperature & Humidity')
    plt.savefig(os.path.join(PLOTS_DIR,'daily_temp_humidity.png'), bbox_inches='tight')
    plt.close()
    return daily, weekly, summary

# ----------------------------
# 5) EDA: correlation & plots
# ----------------------------
def eda_plots(df):
    # Correlation matrix
    corr = df[['temperature','humidity','soil_moisture','co2','light']].corr()
    matplt.figure(figsize=(8,6))
    sns.heatmap(corr, annot=True, fmt=".2f", cmap='vlag')
    matplt.title('Sensor Correlation Matrix')
    matplt.savefig(os.path.join(PLOTS_DIR,'correlation_matrix.png'), bbox_inches='tight')
    matplt.close()

    # Time series interactive-ready export (plotly optional in notebook)
    # Save small sample plots
    df[['temperature']].plot(figsize=(12,3), title='Temperature (Hourly)')
    matplt.savefig(os.path.join(PLOTS_DIR,'temperature_ts.png'), bbox_inches='tight')
    matplt.close()

    return corr

# ----------------------------
# 6) Define ideal ranges & KPIs
# ----------------------------
def compute_kpis(df, ideal_ranges=IDEAL_RANGES):
    # Binary 'in_ideal' columns
    for var, (lo, hi) in ideal_ranges.items():
        if var in df.columns:
            col = f'{var}_ok'
            df[col] = df[var].between(lo, hi)

    # Compute percent of time per day in ideal range for each metric
    kpi_daily = {}
    for var in ideal_ranges.keys():
        col = f'{var}_ok'
        if col in df.columns:
            pct = df[col].resample('D').mean() * 100
            kpi_daily[var] = pct
            # Save a plot per KPI
            matplt.figure(figsize=(10,2))
            pct.plot(title=f'%time in ideal range per day: {var}')
            matplt.ylabel('%')
            matplt.savefig(os.path.join(PLOTS_DIR,f'kpi_{var}_daily.png'), bbox_inches='tight')
            matplt.close()
    return df, kpi_daily

# ----------------------------
# 7) Anomaly detection
# ----------------------------
def detect_anomalies(df):
    # RULE-BASED anomalies
    df['temp_delta'] = df['temperature'].diff().abs()
    df['anomaly_rule'] = (df['temperature'] > 35) | (df['temp_delta'] > 8)

    # Z-SCORE anomalies for temperature (abs z > 3)
    df['temp_z'] = (df['temperature'] - df['temperature'].mean()) / df['temperature'].std()
    df['anomaly_z'] = df['temp_z'].abs() > 3

    # Isolation Forest (multivariate)
    features = ['temperature','humidity','soil_moisture','co2','light']
    feat_df = df[features].fillna(method='ffill').fillna(method='bfill')
    iso = IsolationForest(contamination=0.01, random_state=42)
    iso_pred = iso.fit_predict(feat_df)
    df['iso_anom'] = (iso_pred == -1)

    # Final anomaly flag (OR) — you could also majority-vote
    df['anomaly_final'] = df[['anomaly_rule','anomaly_z','iso_anom']].any(axis=1)

    # Export anomalies with context
    anoms = df[df['anomaly_final']].copy()
    anoms['matched_rules'] = (
        anoms[['anomaly_rule','anomaly_z','iso_anom']]
        .apply(lambda row: ','.join([c for c,v in zip(['rule','zscore','iso'], row) if v]), axis=1)
    )
    anoms.to_csv(ANOMALIES_PATH)
    print(f"Detected {len(anoms)} anomalies. Saved to {ANOMALIES_PATH}")
    return df, anoms

# ----------------------------
# 8) Relationship & cross-correlation checks
# ----------------------------
def relationship_checks(df, var_x='light', var_y='temperature', nlags=48):
    # Fill NaNs for ccf calculation
    x = df[var_x].fillna(0).values
    y = df[var_y].fillna(0).values
    # compute cross-correlation function (note: ccf returns non-negative lags correlation)
    try:
        cc = ccf(x, y)[:nlags]
        # save small plot of cross-correlation
        matplt.figure(figsize=(10,3))
        matplt.stem(range(len(cc)), cc, use_line_collection=True)
        matplt.title(f'Cross-correlation: {var_x} -> {var_y}')
        matplt.xlabel('Lag')
        matplt.savefig(os.path.join(PLOTS_DIR,f'ccf_{var_x}_{var_y}.png'), bbox_inches='tight')
        matplt.close()
        return cc
    except Exception as e:
        print('Cross-correlation failed:', e)
        return None

# ----------------------------
# 9) Forecasting: Prophet if available, else RandomForest
# ----------------------------
def forecast_temperature(df, horizon_hours=24*7):
    # prepare series of temperature
    temp = df['temperature'].reset_index().rename(columns={'timestamp':'ds','temperature':'y'})
    temp = temp.dropna()

    if PROPHET_AVAILABLE:
        print("Using Prophet for forecasting.")
        m = Prophet(daily_seasonality=True, weekly_seasonality=True)
        m.fit(temp)
        future = m.make_future_dataframe(periods=horizon_hours, freq='H')
        fcst = m.predict(future)
        fcst_out = fcst[['ds','yhat','yhat_lower','yhat_upper']].rename(columns={'yhat':'yhat_temp'})
        fcst_out.to_csv(FORECAST_PATH, index=False)
        joblib.dump(m, MODEL_PATH)
        print('Prophet forecast saved to', FORECAST_PATH)
        return fcst_out
    else:
        # RandomForest time-series style forecasting using lag features
        print("Prophet not available — using RandomForest fallback.")
        df_rf = df[['temperature']].copy()
        # create lag features
        for lag in [1,3,6,12,24,48]:
            df_rf[f'temp_lag_{lag}'] = df_rf['temperature'].shift(lag)
        # rolling features
        df_rf['temp_roll_24'] = df_rf['temperature'].rolling(24).mean().shift(1)
        df_rf = df_rf.dropna()

        # predict next 24*horizon as single-step iterative forecast
        # split train/test by time
        train = df_rf.iloc[:-horizon_hours]
        test = df_rf.iloc[-horizon_hours:]

        X_train = train.drop(columns=['temperature'])
        y_train = train['temperature']
        X_test = test.drop(columns=['temperature'])
        y_test = test['temperature']

        rf = RandomForestRegressor(n_estimators=200, random_state=42)
        rf.fit(X_train, y_train)
        preds = rf.predict(X_test)

        # Evaluate
        mae = mean_absolute_error(y_test, preds)
        rmse = mean_squared_error(y_test, preds, squared=False)
        r2 = r2_score(y_test, preds)
        print(f"RF Forecast eval — MAE: {mae:.3f}, RMSE: {rmse:.3f}, R2: {r2:.3f}")

        # Build forecast output aligned with test index
        fcst_out = pd.DataFrame({
            'ds': X_test.index,
            'yhat_temp': preds
        })
        fcst_out.to_csv(FORECAST_PATH, index=False)
        joblib.dump(rf, MODEL_PATH)
        print('RandomForest forecast saved to', FORECAST_PATH)
        return fcst_out

# ----------------------------
# 10) Write a short findings report skeleton
# ----------------------------
def write_report(df, anoms, forecast):
    lines = []
    lines.append('# Greenhouse Analytics — Findings (AUTO-GENERATED)')
    lines.append('')
    lines.append('## Data summary')
    lines.append(f'- Data period: {df.index.min()} to {df.index.max()}')
    lines.append(f'- Total datapoints (hourly): {len(df)}')
    lines.append('')
    lines.append('## KPI (time in ideal ranges) — latest 7 days averages')
    for var in IDEAL_RANGES.keys():
        col = f'{var}_ok'
        if col in df.columns:
            pct = df[col].resample('D').mean().tail(7).mean() * 100
            lines.append(f'- {var}: {pct:.1f}% time in ideal range (last 7 days)')
    lines.append('')
    lines.append('## Anomalies')
    lines.append(f'- Total anomalies detected: {len(anoms)}')
    lines.append('- Sample anomalies (timestamp, temperature, matched_rules):')
    for idx, row in anoms.head(5).iterrows():
        lines.append(f'  - {idx} | temp={row.get("temperature")} | rules={row.get("matched_rules")}')
    lines.append('')
    lines.append('## Forecast (next period)')
    lines.append(f'- Forecast rows saved to: {FORECAST_PATH}')
    lines.append('')
    lines.append('## Recommendations')
    lines.append('- Investigate repeated anomalies at same times — could be sensor drift or scheduled events.')
    lines.append('- If temperature frequently exceeds ideal max, consider improving ventilation / shading.')
    lines.append('- Use the forecast to preemptively adjust HVAC/irrigation schedules.')
    with open(REPORT_PATH, 'w') as f:
        f.write('\n'.join(lines))
    print('Report skeleton written to', REPORT_PATH)

# ----------------------------
# 11) Main pipeline orchestration
# ----------------------------
# Define the path to your data file
RAW_PATH = 'raw/greenhouse_raw.csv'  # This path needs to be updated

# Make sure the file exists before running the code
import os

def main():
    # Check if the file exists
    if not os.path.exists(RAW_PATH):
        print(f"Error: The file {RAW_PATH} does not exist.")
        print("Please ensure the file exists or update the RAW_PATH variable.")
        return  # Exit the function if file doesn't exist
    
    df_raw = ingest(RAW_PATH)
    df_clean = clean_and_resample(df_raw)
    daily, weekly, summary = aggregates_and_summary(df_clean)
    corr = eda_plots(df_clean)
    df_with_kpis, kpi_daily = compute_kpis(df_clean)
    df_anom, anoms = detect_anomalies(df_with_kpis)
    cc = relationship_checks(df_anom, var_x='light', var_y='temperature', nlags=48)
    forecast = forecast_temperature(df_anom, horizon_hours=24*7)
    write_report(df_anom, anoms, forecast)
    print('Pipeline complete. Outputs in /data, /models, /images, /report.')

if __name__ == '__main__':
    main()


Ingesting data from raw/greenhouse_raw.csv
Rows: 720
Saved cleaned data to data/greenhouse_cleaned.csv
Detected 8 anomalies. Saved to data/anomalies.csv
Cross-correlation failed: stem() got an unexpected keyword argument 'use_line_collection'
Prophet not available — using RandomForest fallback.
RF Forecast eval — MAE: 0.516, RMSE: 0.653, R2: 0.983
RandomForest forecast saved to data/temperature_forecast.csv
Report skeleton written to report/findings.md
Pipeline complete. Outputs in /data, /models, /images, /report.


<Figure size 1000x600 with 0 Axes>

<Figure size 1000x300 with 0 Axes>